In [1]:
import sys
import gym
import numpy as np
import random
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

c:\users\dq315\appdata\local\continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class DeepQNetwork:
    def __init__(self, sess, n_actions, n_features, learning_rate, gamma, replace_target_iter):
        self.sess = sess
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = gamma
        self.learn_step_counter = 0
        self.replace_target_iter = replace_target_iter
        
        # inputs 
        self.S = tf.placeholder(tf.float32, [None, self.n_features], name='s')  
        self.S_ = tf.placeholder(tf.float32, [None, self.n_features], name='s_')    
        self.A = tf.placeholder(tf.int32, [None, ], name='a')
        self.R = tf.placeholder(tf.float32, [None, ], name='r')
        self.D = tf.placeholder(tf.bool, [None, ], name='done')
        
        # variables
        with tf.variable_scope('net'):
            self.q = self._build_net(self.S, scope='eval', trainable=True)
            self.q_ = self._build_net(self.S_, scope='target', trainable=False)
        
        # parameters for target_net and evaluate_net
        self.e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='net/eval')
        self.t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='net/target')
        
        # target_net hard replacement
        self.target_replace = [tf.assign(t, e) for t, e in zip(self.t_params, self.e_params)]
        
        # step 1: compute q'(s',a') >> s' from batch, a' from target_net (amax)
        q_target = self.R if self.D is True else self.R + self.gamma * tf.reduce_max(self.q_, axis=1) 
        # step 2: compute q(s,a) >> s from batch, a from batch
        a_indices = tf.stack([tf.range(batch_size), self.A], axis=1)
        q_pred = tf.gather_nd(params=self.q, indices=a_indices) 
        # step 3: compute td_error & train 
        self.td_error = tf.losses.mean_squared_error(labels=q_target, predictions=q_pred)
        self.train = tf.train.AdamOptimizer(self.lr).minimize(self.td_error, var_list=self.e_params) 
        
    def _build_net(self, s, scope, trainable):
        with tf.variable_scope(scope):
            init_w = tf.contrib.layers.xavier_initializer()
            init_b = tf.constant_initializer(0.1)
            net = tf.layers.dense(s, 24, tf.nn.relu, 
                                  kernel_initializer=init_w, bias_initializer=init_b, 
                                  name='l1', trainable=trainable)
            net = tf.layers.dense(net, 24, tf.nn.relu, 
                                  kernel_initializer=init_w, bias_initializer=init_b, 
                                  name='l2', trainable=trainable)
            q = tf.layers.dense(net, self.n_actions, 
                                kernel_initializer=init_w, bias_initializer=init_b, 
                                name='q', trainable=trainable)
            return q

    def choose_action(self, s):
        if np.random.uniform() < epsilon:
            action = random.randrange(self.n_actions)
        else:
            actions_value = self.sess.run(self.q, feed_dict={self.S: s[np.newaxis, :]})
            action = np.argmax(actions_value)
        return action

    def learn(self, bs, ba, br, bs_, bd):
        # train evaluate_net and get loss
        self.sess.run(self.train, feed_dict={self.S: bs, self.A: ba, self.R: br, self.S_: bs_, self.D: bd})        
        l = self.sess.run(self.td_error, feed_dict={self.S: bs, self.A: ba, self.R: br, self.S_: bs_, self.D: bd})
        self.learn_step_counter += 1
        return l
    
    def update_target(self):
        # check to replace target parameters
        if self.learn_step_counter % self.replace_target_iter == 0:
            self.sess.run(self.target_replace)

In [3]:
class Memory(object):
    def __init__(self, capacity, dims):
        self.capacity = capacity
        self.memory = np.zeros((capacity, dims))
        self.counter = 0

    def store_transition(self, s, a, r, s_, done):
        transition = np.hstack((s, [a], [r], s_, [done]))
        index = self.counter % self.capacity  
        self.memory[index, :] = transition
        self.counter += 1

    def sample(self, n):
        indices = np.random.choice(np.minimum(self.capacity, self.counter), size=n)
        return self.memory[indices, :]

In [ ]:
env = gym.make('CartPole-v1')    # In case of CartPole-v1, maximum length of episode is 500

np.random.seed(1)
tf.set_random_seed(1)

# network parameters
n_actions = env.action_space.n
n_features = env.observation_space.shape[0]
print('n_features:', n_features)
print('n_actions:', n_actions)
learning_rate = 0.001
gamma = 0.99
replace_target_iter = 1

# exploration parameters
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.999
        
# reply buffer parameters
capacity = 2000
train_start = 1000
batch_size = 64
dims = n_features * 2 + 1 + 1 + 1
print('batch dims:', dims)

EPISODES = 200

# dqn agent & reply buffer
sess = tf.Session()
dqn = DeepQNetwork(sess, n_actions, n_features, learning_rate, gamma, replace_target_iter)
M = Memory(capacity, dims)
sess.run(tf.global_variables_initializer())

n_features: 4
n_actions: 2
batch dims: 11


In [ ]:
episode_r = []
cost_his = []

for episode in range(EPISODES):
    done = False
    step_count = 0
    step_r = 0
    step_loss = 0

    # initial observation
    state = env.reset()

    while not done:
        
        step_count += 1

        # fresh env
        if True:
            env.render()

        # RL choose action based on state
        action = dqn.choose_action(state)

        # RL take action and get next state and reward
        next_state, reward, done, info = env.step(action)
        # CartPole-v1 >> if an action make the episode end, then gives penalty of -100
        reward = reward if not done or step_r == 499 else -100
        
        # store <s, a, r, s_>
        M.store_transition(state, action, reward, next_state, done)

        # train eval_net, epsilon decay, update target_net
        if M.counter >= train_start:

            # sample batch
            bt = M.sample(batch_size)
            bs = bt[:, :n_features]
            ba = bt[:, n_features: n_features + 1]
            br = bt[:, -n_features - 2: -n_features -1]
            bs_ = bt[:, -n_features -1: -1]
            bd = bt[:, -1:]           
            ba = np.hstack(ba).astype(int) #(batch_size,)
            br = np.hstack(br) #(batch_size,)
            bd = np.hstack(bd) #(batch_size,)
            # train
            tra_results = dqn.learn(bs, ba, br, bs_, bd)
            step_loss += tra_results
            cost_his.append(step_loss)
            
            # epsilon decay per train
            if epsilon > epsilon_min:
                epsilon *= epsilon_decay
            
            # update target network every C steps
            #dqn.update_target()

        # swap observation
        state = next_state

        # record step reward
        step_r += reward

        # break while loop when end of this episode
        if done:
            
            # update target network at the end of the epsiode game has a better performance
            dqn.update_target()
            
            step_r = step_r if step_r == 500 else step_r + 100
            episode_r.append(step_r)

            print('Episode:', episode+1, 
                  'Reward:', step_r,
                  'Loss:', '%.0f'%(step_loss/step_count),
                  'Epsilon:', '%.4f'%epsilon,)        

Episode: 1 Reward: 57.0 Loss: 0 Epsilon: 1.0000
Episode: 2 Reward: 13.0 Loss: 0 Epsilon: 1.0000
Episode: 3 Reward: 82.0 Loss: 0 Epsilon: 1.0000
Episode: 4 Reward: 9.0 Loss: 0 Epsilon: 1.0000
Episode: 5 Reward: 15.0 Loss: 0 Epsilon: 1.0000
Episode: 6 Reward: 14.0 Loss: 0 Epsilon: 1.0000
Episode: 7 Reward: 16.0 Loss: 0 Epsilon: 1.0000
Episode: 8 Reward: 17.0 Loss: 0 Epsilon: 1.0000
Episode: 9 Reward: 11.0 Loss: 0 Epsilon: 1.0000
Episode: 10 Reward: 19.0 Loss: 0 Epsilon: 1.0000
Episode: 11 Reward: 15.0 Loss: 0 Epsilon: 1.0000
Episode: 12 Reward: 14.0 Loss: 0 Epsilon: 1.0000
Episode: 13 Reward: 18.0 Loss: 0 Epsilon: 1.0000
Episode: 14 Reward: 23.0 Loss: 0 Epsilon: 1.0000
Episode: 15 Reward: 30.0 Loss: 0 Epsilon: 1.0000
Episode: 16 Reward: 9.0 Loss: 0 Epsilon: 1.0000
Episode: 17 Reward: 33.0 Loss: 0 Epsilon: 1.0000
Episode: 18 Reward: 21.0 Loss: 0 Epsilon: 1.0000
Episode: 19 Reward: 15.0 Loss: 0 Epsilon: 1.0000
Episode: 20 Reward: 50.0 Loss: 0 Epsilon: 1.0000
Episode: 21 Reward: 64.0 Loss: 

In [ ]:
plt.figure(figsize=(25, 8))
plt.plot(np.arange(len(episode_r)), episode_r)
plt.ylabel('Reward',fontsize=20)
plt.xlabel('training episodes',fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(25, 8))
plt.plot(np.arange(len(cost_his)), cost_his)
plt.ylabel('Loss',fontsize=20)
plt.xlabel('training steps',fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

In [1]:
import numpy as np

arr = np.arange(12).reshape(4, 3)
print('arr:', arr)
print('')

a = np.arange(4)
print('a:', a)
print('')

b = np.array([1,1,2,2])
print('b:', b)
print('')

x = arr[a,b]
print(x)

arr: [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]

a: [0 1 2 3]

b: [1 1 2 2]

[ 1  4  8 11]


In [2]:
import tensorflow as tf
import numpy as np

sess = tf.Session()

arr = np.arange(12).reshape(4, 3)
print('arr:', arr)
print('')

a = np.arange(4)
print('a:', (a))
print('')

b = np.array([1,1,2,2])
print('b:', b)
print('')

x = arr[a,b]
print('x:', x)

c:\users\dq315\appdata\local\continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


arr: [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]

a: [0 1 2 3]

b: [1 1 2 2]

x: [ 1  4  8 11]


In [12]:
import tensorflow as tf
import numpy as np

sess = tf.Session()

arr = np.arange(12).reshape(4, 3)
print('arr:', arr)
print('')

a = tf.range(4, dtype=tf.int32)
print('a:', (sess.run(a)))
print('')

b = np.array([1,1,2,2])
print('b:', b)
print('')

x = arr[a,b]
print('x:', x)
print('x:', sess.run(x))

arr: [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]

a: [0 1 2 3]

b: [1 1 2 2]



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices